In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns


In [225]:
data=pd.read_csv('dirty_cafe_sales.csv')

In [226]:
data.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [227]:
print("\nShape of the dataset:", data.shape)


Shape of the dataset: (10000, 8)


In [228]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [229]:
data.isnull().sum()

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

In [231]:
cols = ['Quantity',  'Price Per Unit', 'Total Spent']

for col in cols:
    data[col] = pd.to_numeric(data[col], errors='coerce')


In [ ]:
missing = ['UNKNOWN', 'ERROR', 'unknown', 'error', 'n/a', 'na', 'null', '']

for col in data.columns:
    if data[col].dtype == 'object':  
        data[col] = data[col].str.strip().str.lower().replace(missing, np.nan)


In [234]:
data.isnull().sum()

Transaction ID         0
Item                 969
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

In [235]:
data.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,txn_1961373,coffee,2.0,2.0,4.0,credit card,takeaway,2023-09-08
1,txn_4977031,cake,4.0,3.0,12.0,cash,in-store,2023-05-16
2,txn_4271903,cookie,4.0,1.0,NaN,credit card,in-store,2023-07-19
3,txn_7034554,salad,2.0,5.0,10.0,NaN,NaN,2023-04-27
4,txn_3160411,coffee,2.0,2.0,4.0,digital wallet,in-store,2023-06-11


In [262]:
replacements = {
    # 3: 'Juice',
    2: 'Coffee',
    1.5: 'Tea',
    4: 'Sandwich',
    5: 'Salad',
    # 3: 'Cake',
    1: 'Cookie',
    4: 'Smoothie',
   
}

for price, item in replacements.items():
    
    mask_item = (data['Price Per Unit'] == price) & (data['Item'].isna())
    data.loc[mask_item, 'Item'] = item
    
    mask_price = (data['Item'] == item) & (data['Price Per Unit'].isna())
    data.loc[mask_price, 'Price Per Unit'] = price


In [263]:
data['Item'].isna().sum()


253

In [264]:
mask = data['Total Spent'].isna()
data.loc[mask, 'Total Spent'] = data.loc[mask, 'Price Per Unit'] * data.loc[mask, 'Quantity']


In [265]:
data['Total Spent'].isna().sum()

40

In [266]:
mask = data['Quantity'].isna()
data.loc[mask, 'Quantity'] = data.loc[mask, 'Total Spent']  / data.loc[mask, 'Price Per Unit']

In [243]:
data['Quantity'].isna().sum()

479

In [267]:
data['Quantity'].isna().sum()

38

In [268]:
mask = data['Price Per Unit'].isna()
data.loc[mask, 'Price Per Unit'] = data.loc[mask, 'Total Spent']  / data.loc[mask, 'Quantity']

In [269]:
data['Price Per Unit'].isna().sum()

38

In [270]:
data['Transaction Date'] = pd.to_datetime(data['Transaction Date'])

data['Day of Week'] = data['Transaction Date'].dt.day_name()
data['Transaction Month'] = data['Transaction Date'].dt.month_name()

print(data[['Transaction Date', 'Day of Week', 'Transaction Month']].head())


  Transaction Date Day of Week Transaction Month
0       2023-09-08      Friday         September
1       2023-05-16     Tuesday               May
2       2023-07-19   Wednesday              July
3       2023-04-27    Thursday             April
4       2023-06-11      Sunday              June


In [271]:
most_product = (
    data.groupby('Item')['Quantity']
    .sum()
    .sort_values(ascending=False)
)

print("Most sold product:")
print(most_product.head(1))


Most sold product:
Item
coffee    3534.0
Name: Quantity, dtype: float64


In [272]:
print("Least sold product:")
print(most_product.tail(1))


Least sold product:
Item
Tea    343.0
Name: Quantity, dtype: float64


In [ ]:
# data['Transaction Date'] = pd.to_datetime(data['Transaction Date'])
# data['Month'] = data['Transaction Date'].dt.month_name()



In [286]:
most_month = (
    data.groupby('Month')['Quantity']
    .sum()
    .sort_values(ascending=False)
)

print("Most selling month:")
print(most_month.head(1))


Most selling month:
Month
October    2534.0
Name: Quantity, dtype: float64
